In [ ]:
#!pip install PyPDF2

In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

In [2]:
# location of the pdf file/files. 
#reader = PdfReader('WEF_The_Global_Cooperation_Barometer_2024.pdf')
reader = PdfReader('2023_GPT4All_Technical_Report.pdf')


In [3]:
#reader

# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

print(len(raw_text))

6021


In [4]:
raw_text[:100]

'GPT4All: Training an Assistant-style Chatbot with Large Scale Data\nDistillation from GPT-3.5-Turbo\nY'

In [7]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 500,
    length_function = len,
)
documents = text_splitter.split_text(raw_text)

In [8]:
# local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"

# # Local PDF file uploads
# if local_path:
#   loader = UnstructuredPDFLoader(file_path=local_path)
#   data = loader.load()
# else:
#   print("Upload a PDF file")

In [9]:
len(documents)

12

In [10]:
documents[0]

'GPT4All: Training an Assistant-style Chatbot with Large Scale Data\nDistillation from GPT-3.5-Turbo\nYuvanesh Anand\nyuvanesh@nomic.aiZach Nussbaum\nzanussbaum@gmail.com\nBrandon Duderstadt\nbrandon@nomic.aiBenjamin Schmidt\nben@nomic.aiAndriy Mulyar\nandriy@nomic.ai\nAbstract\nThis preliminary technical report describes the\ndevelopment of GPT4All, a chatbot trained\nover a massive curated corpus of assistant in-\nteractions including word problems, story de-\nscriptions, multi-turn dialogue, and code. We\nopenly release the collected data, data cura-\ntion procedure, training code, and final model\nweights to promote open research and repro-\nducibility. Additionally, we release quantized\n4-bit versions of the model allowing virtually\nanyone to run the model on CPU.\n1 Data Collection and Curation\nWe collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gathered a diverse sam-'

In [11]:
documents[1]

'openly release the collected data, data cura-\ntion procedure, training code, and final model\nweights to promote open research and repro-\nducibility. Additionally, we release quantized\n4-bit versions of the model allowing virtually\nanyone to run the model on CPU.\n1 Data Collection and Curation\nWe collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gathered a diverse sam-\nple of questions/prompts by leveraging three pub-\nlicly available datasets:\n• The unified chip2 subset of LAION OIG.\n• Coding questions with a random sub-sample\nof Stackoverflow Questions\n• Instruction-tuning with a sub-sample of Big-\nscience/P3\nWe chose to dedicate substantial attention to data\npreparation and curation based on commentary in\nthe Stanford Alpaca project (Taori et al., 2023).\nUpon collection of the initial dataset of prompt-\ngeneration pairs, we loaded data into Atlas for data'

In [14]:
# !pip install --q chromadb
# !pip install --q langchain-text-splitters

In [15]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [16]:
#!pip install sentence-transformers

In [17]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
#EMBED_MODEL = "multi-qa-MiniLM-L6-cos-v1" 

COLLECTION_NAME = "GPT4_tech_report-1" #chunk-1000 & overlap - 500
# "GPT4_tech_report-1"  #chunk-200 & overlap - 20 - gave poor results

client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [18]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
     model_name=EMBED_MODEL
 )

collection = client.create_collection(
     name=COLLECTION_NAME,
     embedding_function=embedding_func,
     metadata={"hnsw:space": "cosine"},
 )

c:\Users\monda\LeetCode\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
collection.add(
    documents=documents,
     ids=[f"id{i}" for i in range(len(documents))]
)

In [20]:
query_results = collection.query(
     query_texts=["How data was collected and curated for GPT4All training?"],
     n_results=1,
 )


In [21]:
query_results.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

In [22]:
query_results['documents']

[['ple of questions/prompts by leveraging three pub-\nlicly available datasets:\n• The unified chip2 subset of LAION OIG.\n• Coding questions with a random sub-sample\nof Stackoverflow Questions\n• Instruction-tuning with a sub-sample of Big-\nscience/P3\nWe chose to dedicate substantial attention to data\npreparation and curation based on commentary in\nthe Stanford Alpaca project (Taori et al., 2023).\nUpon collection of the initial dataset of prompt-\ngeneration pairs, we loaded data into Atlas for data\ncuration and cleaning. With Atlas, we removed all\nexamples where GPT-3.5-Turbo failed to respond\nto prompts and produced malformed output. This\nreduced our total number of examples to 806,199\nhigh-quality prompt-generation pairs. Next, we\ndecided to remove the entire Bigscience/P3 sub-\nset from the final training dataset due to its very\nFigure 1: TSNE visualization of the candidate training\ndata (Red: Stackoverflow, Orange: chip2, Blue: P3).\nThe large blue balls (e.g. indic

In [23]:
query_results["distances"]

[[0.5216152021796041]]

In [24]:
query_results2 = collection.query(
     query_texts=["who are the authors of the article?"],
     n_results=1,
 )


In [25]:
query_results2['documents'][0]


['license. The assistant data is gathered from Ope-\nnAI’s GPT-3.5-Turbo, whose terms of use pro-hibit developing models that compete commer-\ncially with OpenAI.\nReferences\nEdward J. Hu, Yelong Shen, Phillip Wallis, Zeyuan\nAllen-Zhu, Yuanzhi Li, Shean Wang, Lu Wang, and\nWeizhu Chen. 2021. Lora: Low-rank adaptation of\nlarge language models.\nRohan Taori, Ishaan Gulrajani, Tianyi Zhang, Yann\nDubois, Xuechen Li, Carlos Guestrin, Percy\nLiang, and Tatsunori B. Hashimoto. 2023. Stan-\nford alpaca: An instruction-following llama\nmodel. https://github.com/tatsu-lab/\nstanford_alpaca .\nHugo Touvron, Thibaut Lavril, Gautier Izacard, Xavier\nMartinet, Marie-Anne Lachaux, Timoth ´ee Lacroix,\nBaptiste Rozi `ere, Naman Goyal, Eric Hambro,\nFaisal Azhar, Aurelien Rodriguez, Armand Joulin,\nEdouard Grave, and Guillaume Lample. 2023.\nLlama: Open and efficient foundation language\nmodels.\nYizhong Wang, Yeganeh Kordi, Swaroop Mishra, Al-\nisa Liu, Noah A. Smith, Daniel Khashabi, and Han-']

In [26]:
#query_results2.keys()
query_results2['distances'][0]


[0.805583207935756]

In [27]:


#query = "the cost of training the GPT4all model?"
#query = "How was the model trained?"
query = "considerations used for GPT4all? "

query_results3 = collection.query(
     query_texts=[query],
     n_results=1,
 )


In [28]:
query_results3['distances']

[[0.6191096918007521]]

In [29]:
query_results3['documents']

[['curation and cleaning. With Atlas, we removed all\nexamples where GPT-3.5-Turbo failed to respond\nto prompts and produced malformed output. This\nreduced our total number of examples to 806,199\nhigh-quality prompt-generation pairs. Next, we\ndecided to remove the entire Bigscience/P3 sub-\nset from the final training dataset due to its very\nFigure 1: TSNE visualization of the candidate training\ndata (Red: Stackoverflow, Orange: chip2, Blue: P3).\nThe large blue balls (e.g. indicated by the red arrow)\nare highly homogeneous prompt-response pairs.\nlow output diversity; P3 contains many homoge-\nneous prompts which produce short and homoge-\nneous responses from GPT-3.5-Turbo. This exclu-\nsion produces a final subset containing 437,605\nprompt-generation pairs, which is visualized in\nFigure 2. You can interactively explore the dataset\nat each stage of cleaning at the following links:\n• Cleaned with P3\n• Cleaned without P3 (Final Training Dataset)\n2 Model Training']]

In [30]:
query = "Summerize the document"
#query = "How was the model trained?"

query_results4 = collection.query(
     query_texts=[query],
     n_results=1,
 )


In [31]:
query_results4['documents']

[['ford alpaca: An instruction-following llama\nmodel. https://github.com/tatsu-lab/\nstanford_alpaca .\nHugo Touvron, Thibaut Lavril, Gautier Izacard, Xavier\nMartinet, Marie-Anne Lachaux, Timoth ´ee Lacroix,\nBaptiste Rozi `ere, Naman Goyal, Eric Hambro,\nFaisal Azhar, Aurelien Rodriguez, Armand Joulin,\nEdouard Grave, and Guillaume Lample. 2023.\nLlama: Open and efficient foundation language\nmodels.\nYizhong Wang, Yeganeh Kordi, Swaroop Mishra, Al-\nisa Liu, Noah A. Smith, Daniel Khashabi, and Han-\nnaneh Hajishirzi. 2022. Self-instruct: Aligning lan-\nguage model with self generated instructions.']]

In [32]:
query_results4['distances']

[[0.8126886227707455]]

In [ ]:
# from sentence_transformers import SentenceTransformer, util

# # Download model
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# # The sentences we'd like to compute similarity about
# sentences = ['Python is an interpreted high-level general-purpose programming language.',
#     'Python is dynamically-typed and garbage-collected.',
#     'The quick brown fox jumps over the lazy dog.']

# # Get embeddings of sentences
# embeddings = model.encode(sentences)

# # Compute similarities
# sim = util.cos_sim(embeddings[0], embeddings[1])
# print("{0:.4f}".format(sim.tolist()[0][0])) # 0.6445
# sim = util.cos_sim(embeddings[0], embeddings[2])
# print("{0:.4f}".format(sim.tolist()[0][0])) # 0.0365